# Draft Assistant

A notebook-based draft assistant.

In [1]:
# Install package
%pip install .. -q

import pandas as pd
import statisticaldrafting as sd

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load a model. 
# sd.list_sets() # Show available sets.
dm = sd.DraftModel(set="FDN", draft_mode="Premier")

In [3]:
# collection = ["Vivien Reid", "Scavenging Ooze"] # Collection as cardnames
# collection = [16, 229, 78, 78, 198] # Collection as card ids
# collection = [66] * 5 + [134] * 5 + [188] * 2 + [169] * 4 + [129] * 4 + [100] * 3 + [214] * 4 # BW FDN deck
collection = []
pick_order = dm.get_pick_order(collection)
# pick_order = pick_order[pick_order["rarity"].isin(["common", "uncommon"])] # Optional filter. 
# pick_order = pick_order[pick_order["color_identity"].isin(["R"])] # Optional filter. 
pick_order

,name,rarity,color_identity,p1p1_rating,synergy,rating
162,"Liliana, Dreadhorde General",mythic,B,95.7,0.0,95.7
63,Curator of Destinies,rare,U,95.2,0.0,95.2
85,Embercleave,special,R,94.9,0.0,94.9
54,"Chandra, Flameshaper",mythic,R,94.2,0.0,94.2
35,Bloodthirsty Conqueror,mythic,B,92.2,0.0,92.2
155,"Kiora, the Rising Tide",rare,U,92.2,0.0,92.2
263,Twinflame Tyrant,mythic,R,91.7,0.0,91.7
52,Celestial Armor,rare,W,91.7,0.0,91.7
15,"Arahbo, the First Fang",rare,W,91.6,0.0,91.6
138,High-Society Hunter,rare,B,91.3,0.0,91.3


In [4]:
# Deckbuild recommendations after draft. 
# dm.get_deck_recommendation(coll]"ection, starting_colors="").head(30)